In [192]:
import sys
sys.path.append('/Users/reginaceballos/Documents/MIT/2023 - Spring/6.8110 - Cognitive Robotics/Final project/cogrob-grand-challenge/')
sys.path.append('/Users/reginaceballos/Documents/MIT/2023 - Spring/6.8110 - Cognitive Robotics/Final project/cogrob-grand-challenge/Benchmark')

In [193]:
from src.propositional_state_logic import *
from src.utils import *
from collections import defaultdict
import itertools
from copy import deepcopy

In [194]:
p = Problem()

# Define the variables above. Returns a Variable object.
F1 = p.add_variable('F1', type='binary')
P1 = p.add_variable('P1', type='binary')
V1 = p.add_variable('V1', type='binary')
P2 = p.add_variable('P2', type='binary')
V2 = p.add_variable('V2', type='binary')
PV2 = p.add_variable('PV2', type='binary')
V3 = p.add_variable('V3', type='binary')
PV3 = p.add_variable('PV3', type='binary')
P3 = p.add_variable('P3', type='binary')
R1 = p.add_variable('R1', type='binary')
T1 = p.add_variable('T1', type='binary')

# Add the word problem constraints.
p.add_constraint('F1 => P1')
p.add_constraint('~F1 => ~P1')

p.add_constraint('V1 & P1 => P2')
p.add_constraint('V1 & ~P1 => ~P2')
p.add_constraint('~V1 => ~P2')

p.add_constraint('V2 & P2 => PV2')
p.add_constraint('V2 & ~P2 => ~PV2')
p.add_constraint('~V2 => ~PV2')

p.add_constraint('V3 & P2 => PV3')
p.add_constraint('V3 & ~P2 => ~PV3')
p.add_constraint('~V3 => ~PV3')

p.add_constraint('~PV2 & ~PV3 => ~P3')
p.add_constraint('PV2 => P3')
p.add_constraint('PV3 => P3')

p.add_constraint('R1 & P3 => T1')
p.add_constraint('R1 & ~P3 => ~T1')
p.add_constraint('~R1 => ~T1')

# Prints out constraints nicely in LaTeX, so you can check them.
display_constraints(p)

Constraints:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [195]:
sat = SATSolver(p)
# Make an assignment to some variables; variable.get_assignment(value) returns an Assignment.
assignment = frozenset([F1.get_assignment('True'), P1.get_assignment('False')])
sat.check_consistency(assignment)

(False, None, frozenset())

In [196]:
Vs = ['T1', 'R1', 'P3']
Vs

['T1', 'R1', 'P3']

In [197]:
#Vs = ['F1', 'P1']
#Vs

In [198]:
#Vs = ['P1', 'V1', 'V2', 'V3', 'P3']
#Vs

In [199]:
def findsubsets(s, n):
    return (list(tup) for tup in itertools.combinations(s, n))

In [200]:
def get_nodes(variable_list):
    nodes = []

    for i in range(len(variable_list)+1):
        for n in findsubsets(variable_list, i):
            nodes.append(n)

    return nodes

In [201]:
all_domains = []

for v in Vs:
    var = p.get_variable(v)
    for a in list(var.domain):
        all_domains.append(var.get_assignment(a))

all_domains

[T1, ¬T1, R1, ¬R1, P3, ¬P3]

In [202]:
full_set = get_nodes(all_domains)
reduced_set = []

for v in full_set:
    if len(v) == len(set([a.var for a in v])):
        #reduced_set.append([a.__str__() for a in v])
        reduced_set.append(v)

reduced_set


[[],
 [T1],
 [¬T1],
 [R1],
 [¬R1],
 [P3],
 [¬P3],
 [T1, R1],
 [T1, ¬R1],
 [T1, P3],
 [T1, ¬P3],
 [¬T1, R1],
 [¬T1, ¬R1],
 [¬T1, P3],
 [¬T1, ¬P3],
 [R1, P3],
 [R1, ¬P3],
 [¬R1, P3],
 [¬R1, ¬P3],
 [T1, R1, P3],
 [T1, R1, ¬P3],
 [T1, ¬R1, P3],
 [T1, ¬R1, ¬P3],
 [¬T1, R1, P3],
 [¬T1, R1, ¬P3],
 [¬T1, ¬R1, P3],
 [¬T1, ¬R1, ¬P3]]

In [203]:
def list_tree(nodes, n):
    new_nodes = deepcopy(nodes)
    if n in new_nodes:
        new_nodes.remove(n)

    a = []
    a.append(n)

    if new_nodes == [] or [j for j in new_nodes if n != j and [v.__str__() for v in n] == [v.__str__() for v in j[:len(n)]] and len(j) == len(n)+1] == []:
        a.append([])
        
    else:
        a.append([list_tree([k for k in new_nodes if k != j], j) for j in new_nodes if n != j and [v.__str__() for v in n] == [v.__str__() for v in j[:len(n)]] and len(j) == len(n)+1])
    
    return a

In [204]:
tree = list_tree(reduced_set, [])
tree

[[],
 [[[T1],
   [[[T1, R1], [[[T1, R1, P3], []], [[T1, R1, ¬P3], []]]],
    [[T1, ¬R1], [[[T1, ¬R1, P3], []], [[T1, ¬R1, ¬P3], []]]],
    [[T1, P3], []],
    [[T1, ¬P3], []]]],
  [[¬T1],
   [[[¬T1, R1], [[[¬T1, R1, P3], []], [[¬T1, R1, ¬P3], []]]],
    [[¬T1, ¬R1], [[[¬T1, ¬R1, P3], []], [[¬T1, ¬R1, ¬P3], []]]],
    [[¬T1, P3], []],
    [[¬T1, ¬P3], []]]],
  [[R1], [[[R1, P3], []], [[R1, ¬P3], []]]],
  [[¬R1], [[[¬R1, P3], []], [[¬R1, ¬P3], []]]],
  [[P3], []],
  [[¬P3], []]]]

In [205]:
tree[0]

[]

In [206]:
def traverse(o):
    if o != []:
        if isinstance(o, list) and len(o) > 1:
            if not isinstance(o[0], list):
                yield o
            else:
                for value in o:
                    #print('value',value, isinstance(value, list))
                    for subvalue in traverse(value):
                        yield subvalue
        else:
            yield o

In [207]:
for assignment in traverse(tree):
    print(assignment)
    candidate = frozenset([p.get_variable(a.var.__str__()).get_assignment(a.val) for a in assignment])
    print(sat.check_consistency(candidate))

[T1]
(True, frozenset({V3, P2, V1, V2, PV2, P3, F1, R1, T1, P1, PV3}), None)
[T1, R1]
(True, frozenset({V3, P2, V1, V2, PV2, P3, F1, R1, T1, P1, PV3}), None)
[T1, R1, P3]
(True, frozenset({V3, P2, V1, V2, PV2, P3, F1, R1, T1, P1, PV3}), None)
[T1, R1, ¬P3]
(False, None, frozenset())
[T1, ¬R1]
(False, None, frozenset())
[T1, ¬R1, P3]
(False, None, frozenset())
[T1, ¬R1, ¬P3]
(False, None, frozenset())
[T1, P3]
(True, frozenset({V3, P2, V1, V2, PV2, P3, F1, R1, T1, P1, PV3}), None)
[T1, ¬P3]
(False, None, frozenset())
[¬T1]
(True, frozenset({V3, ¬T1, ¬V2, P2, V1, P3, PV3, F1, ¬R1, P1, ¬PV2}), None)
[¬T1, R1]
(True, frozenset({¬PV3, ¬T1, ¬V3, V1, V2, ¬P1, ¬F1, R1, ¬P3, ¬P2, ¬PV2}), None)
[¬T1, R1, P3]
(False, None, frozenset())
[¬T1, R1, ¬P3]
(True, frozenset({¬PV3, ¬T1, ¬V3, V1, V2, ¬P1, ¬F1, R1, ¬P3, ¬P2, ¬PV2}), None)
[¬T1, ¬R1]
(True, frozenset({V3, ¬T1, ¬V2, P2, V1, P3, PV3, F1, ¬R1, P1, ¬PV2}), None)
[¬T1, ¬R1, P3]
(True, frozenset({V3, ¬T1, P2, V1, V2, PV2, P3, F1, ¬R1, P1, PV3}), 

In [208]:
conflicts = []
satisfiable = []

for assignment in traverse(tree):
    candidate = frozenset([p.get_variable(a.var.__str__()).get_assignment(a.val) for a in assignment])
    if sat.check_consistency(candidate)[0] == True:
        satisfiable.append(candidate)
    else:
        conflicts.append(candidate)

print('conflicts',conflicts)
print('satisfiable',satisfiable)


conflicts [frozenset({R1, T1, ¬P3}), frozenset({¬R1, T1}), frozenset({P3, ¬R1, T1}), frozenset({¬R1, T1, ¬P3}), frozenset({T1, ¬P3}), frozenset({P3, R1, ¬T1})]
satisfiable [frozenset({T1}), frozenset({R1, T1}), frozenset({P3, R1, T1}), frozenset({P3, T1}), frozenset({¬T1}), frozenset({R1, ¬T1}), frozenset({R1, ¬T1, ¬P3}), frozenset({¬T1, ¬R1}), frozenset({P3, ¬T1, ¬R1}), frozenset({¬T1, ¬R1, ¬P3}), frozenset({P3, ¬T1}), frozenset({¬T1, ¬P3}), frozenset({R1}), frozenset({P3, R1}), frozenset({R1, ¬P3}), frozenset({¬R1}), frozenset({P3, ¬R1}), frozenset({¬R1, ¬P3}), frozenset({P3}), frozenset({¬P3})]


In [209]:
tree

[[],
 [[[T1],
   [[[T1, R1], [[[T1, R1, P3], []], [[T1, R1, ¬P3], []]]],
    [[T1, ¬R1], [[[T1, ¬R1, P3], []], [[T1, ¬R1, ¬P3], []]]],
    [[T1, P3], []],
    [[T1, ¬P3], []]]],
  [[¬T1],
   [[[¬T1, R1], [[[¬T1, R1, P3], []], [[¬T1, R1, ¬P3], []]]],
    [[¬T1, ¬R1], [[[¬T1, ¬R1, P3], []], [[¬T1, ¬R1, ¬P3], []]]],
    [[¬T1, P3], []],
    [[¬T1, ¬P3], []]]],
  [[R1], [[[R1, P3], []], [[R1, ¬P3], []]]],
  [[¬R1], [[[¬R1, P3], []], [[¬R1, ¬P3], []]]],
  [[P3], []],
  [[¬P3], []]]]

In [210]:
depth = 0

In [211]:
tree[1]

[[[T1],
  [[[T1, R1], [[[T1, R1, P3], []], [[T1, R1, ¬P3], []]]],
   [[T1, ¬R1], [[[T1, ¬R1, P3], []], [[T1, ¬R1, ¬P3], []]]],
   [[T1, P3], []],
   [[T1, ¬P3], []]]],
 [[¬T1],
  [[[¬T1, R1], [[[¬T1, R1, P3], []], [[¬T1, R1, ¬P3], []]]],
   [[¬T1, ¬R1], [[[¬T1, ¬R1, P3], []], [[¬T1, ¬R1, ¬P3], []]]],
   [[¬T1, P3], []],
   [[¬T1, ¬P3], []]]],
 [[R1], [[[R1, P3], []], [[R1, ¬P3], []]]],
 [[¬R1], [[[¬R1, P3], []], [[¬R1, ¬P3], []]]],
 [[P3], []],
 [[¬P3], []]]

In [212]:
reduced_set

[[],
 [T1],
 [¬T1],
 [R1],
 [¬R1],
 [P3],
 [¬P3],
 [T1, R1],
 [T1, ¬R1],
 [T1, P3],
 [T1, ¬P3],
 [¬T1, R1],
 [¬T1, ¬R1],
 [¬T1, P3],
 [¬T1, ¬P3],
 [R1, P3],
 [R1, ¬P3],
 [¬R1, P3],
 [¬R1, ¬P3],
 [T1, R1, P3],
 [T1, R1, ¬P3],
 [T1, ¬R1, P3],
 [T1, ¬R1, ¬P3],
 [¬T1, R1, P3],
 [¬T1, R1, ¬P3],
 [¬T1, ¬R1, P3],
 [¬T1, ¬R1, ¬P3]]

In [213]:
conflicts = []
satisfiable = []

for assignment in reduced_set:
    candidate = frozenset([p.get_variable(a.var.__str__()).get_assignment(a.val) for a in assignment])
    if sat.check_consistency(candidate)[0] == True:
        satisfiable.append(candidate)
    else:
        conflicts.append(candidate)

print('conflicts',conflicts)
print('satisfiable',satisfiable)



conflicts [frozenset({¬R1, T1}), frozenset({T1, ¬P3}), frozenset({R1, T1, ¬P3}), frozenset({P3, ¬R1, T1}), frozenset({¬R1, T1, ¬P3}), frozenset({P3, R1, ¬T1})]
satisfiable [frozenset(), frozenset({T1}), frozenset({¬T1}), frozenset({R1}), frozenset({¬R1}), frozenset({P3}), frozenset({¬P3}), frozenset({R1, T1}), frozenset({P3, T1}), frozenset({R1, ¬T1}), frozenset({¬T1, ¬R1}), frozenset({P3, ¬T1}), frozenset({¬T1, ¬P3}), frozenset({P3, R1}), frozenset({R1, ¬P3}), frozenset({P3, ¬R1}), frozenset({¬R1, ¬P3}), frozenset({P3, R1, T1}), frozenset({R1, ¬T1, ¬P3}), frozenset({P3, ¬T1, ¬R1}), frozenset({¬T1, ¬R1, ¬P3})]


In [214]:
reduced_set_string = []

for v in full_set:
    if len(v) == len(set([a.var for a in v])):
        reduced_set_string.append([a.__str__() for a in v])

reduced_set_string


[[],
 ['T1'],
 ['¬T1'],
 ['R1'],
 ['¬R1'],
 ['P3'],
 ['¬P3'],
 ['T1', 'R1'],
 ['T1', '¬R1'],
 ['T1', 'P3'],
 ['T1', '¬P3'],
 ['¬T1', 'R1'],
 ['¬T1', '¬R1'],
 ['¬T1', 'P3'],
 ['¬T1', '¬P3'],
 ['R1', 'P3'],
 ['R1', '¬P3'],
 ['¬R1', 'P3'],
 ['¬R1', '¬P3'],
 ['T1', 'R1', 'P3'],
 ['T1', 'R1', '¬P3'],
 ['T1', '¬R1', 'P3'],
 ['T1', '¬R1', '¬P3'],
 ['¬T1', 'R1', 'P3'],
 ['¬T1', 'R1', '¬P3'],
 ['¬T1', '¬R1', 'P3'],
 ['¬T1', '¬R1', '¬P3']]

In [215]:
children = {}

for x in reduced_set_string:
    for y in reduced_set_string:
        if x != [] and x != y and all(a in y for a in x):
            if frozenset(x) not in children:
                children[frozenset(x)] = []
                
            children[frozenset(x)].append(y)

children

{frozenset({'T1'}): [['T1', 'R1'],
  ['T1', '¬R1'],
  ['T1', 'P3'],
  ['T1', '¬P3'],
  ['T1', 'R1', 'P3'],
  ['T1', 'R1', '¬P3'],
  ['T1', '¬R1', 'P3'],
  ['T1', '¬R1', '¬P3']],
 frozenset({'¬T1'}): [['¬T1', 'R1'],
  ['¬T1', '¬R1'],
  ['¬T1', 'P3'],
  ['¬T1', '¬P3'],
  ['¬T1', 'R1', 'P3'],
  ['¬T1', 'R1', '¬P3'],
  ['¬T1', '¬R1', 'P3'],
  ['¬T1', '¬R1', '¬P3']],
 frozenset({'R1'}): [['T1', 'R1'],
  ['¬T1', 'R1'],
  ['R1', 'P3'],
  ['R1', '¬P3'],
  ['T1', 'R1', 'P3'],
  ['T1', 'R1', '¬P3'],
  ['¬T1', 'R1', 'P3'],
  ['¬T1', 'R1', '¬P3']],
 frozenset({'¬R1'}): [['T1', '¬R1'],
  ['¬T1', '¬R1'],
  ['¬R1', 'P3'],
  ['¬R1', '¬P3'],
  ['T1', '¬R1', 'P3'],
  ['T1', '¬R1', '¬P3'],
  ['¬T1', '¬R1', 'P3'],
  ['¬T1', '¬R1', '¬P3']],
 frozenset({'P3'}): [['T1', 'P3'],
  ['¬T1', 'P3'],
  ['R1', 'P3'],
  ['¬R1', 'P3'],
  ['T1', 'R1', 'P3'],
  ['T1', '¬R1', 'P3'],
  ['¬T1', 'R1', 'P3'],
  ['¬T1', '¬R1', 'P3']],
 frozenset({'¬P3'}): [['T1', '¬P3'],
  ['¬T1', '¬P3'],
  ['R1', '¬P3'],
  ['¬R1', '¬P3'],
  

In [216]:
reduced_set

[[],
 [T1],
 [¬T1],
 [R1],
 [¬R1],
 [P3],
 [¬P3],
 [T1, R1],
 [T1, ¬R1],
 [T1, P3],
 [T1, ¬P3],
 [¬T1, R1],
 [¬T1, ¬R1],
 [¬T1, P3],
 [¬T1, ¬P3],
 [R1, P3],
 [R1, ¬P3],
 [¬R1, P3],
 [¬R1, ¬P3],
 [T1, R1, P3],
 [T1, R1, ¬P3],
 [T1, ¬R1, P3],
 [T1, ¬R1, ¬P3],
 [¬T1, R1, P3],
 [¬T1, R1, ¬P3],
 [¬T1, ¬R1, P3],
 [¬T1, ¬R1, ¬P3]]

In [217]:
children = {}

for x in reduced_set:
    for y in reduced_set:
        if x != [] and [v.__str__() for v in x] != [v.__str__() for v in y] and all(a in [v.__str__() for v in y] for a in [v.__str__() for v in x]):
            if frozenset(x) not in children:
                children[frozenset(x)] = []
                
            children[frozenset(x)].append(frozenset(y))

children

{frozenset({T1}): [frozenset({R1, T1}),
  frozenset({T1, ¬R1}),
  frozenset({P3, T1}),
  frozenset({T1, ¬P3}),
  frozenset({P3, R1, T1}),
  frozenset({R1, T1, ¬P3}),
  frozenset({P3, T1, ¬R1}),
  frozenset({T1, ¬P3, ¬R1})],
 frozenset({¬T1}): [frozenset({R1, ¬T1}),
  frozenset({¬R1, ¬T1}),
  frozenset({P3, ¬T1}),
  frozenset({¬P3, ¬T1}),
  frozenset({P3, R1, ¬T1}),
  frozenset({R1, ¬P3, ¬T1}),
  frozenset({P3, ¬R1, ¬T1}),
  frozenset({¬P3, ¬R1, ¬T1})],
 frozenset({R1}): [frozenset({R1, T1}),
  frozenset({R1, ¬T1}),
  frozenset({P3, R1}),
  frozenset({R1, ¬P3}),
  frozenset({P3, R1, T1}),
  frozenset({R1, T1, ¬P3}),
  frozenset({P3, R1, ¬T1}),
  frozenset({R1, ¬P3, ¬T1})],
 frozenset({¬R1}): [frozenset({T1, ¬R1}),
  frozenset({¬R1, ¬T1}),
  frozenset({P3, ¬R1}),
  frozenset({¬P3, ¬R1}),
  frozenset({P3, T1, ¬R1}),
  frozenset({T1, ¬P3, ¬R1}),
  frozenset({P3, ¬R1, ¬T1}),
  frozenset({¬P3, ¬R1, ¬T1})],
 frozenset({P3}): [frozenset({P3, T1}),
  frozenset({P3, ¬T1}),
  frozenset({P3, R1}),

In [218]:
def is_conflict(sat, candidate):
    return not sat.check_consistency(candidate)[0]

In [219]:
def is_valid_benchmark(candidate, sat):
    childCandidateList = children[(candidate)]
    for child in childCandidateList:
        if is_conflict(sat, child):
            return False
    return True


In [220]:
reduced_set

[[],
 [T1],
 [¬T1],
 [R1],
 [¬R1],
 [P3],
 [¬P3],
 [T1, R1],
 [T1, ¬R1],
 [T1, P3],
 [T1, ¬P3],
 [¬T1, R1],
 [¬T1, ¬R1],
 [¬T1, P3],
 [¬T1, ¬P3],
 [R1, P3],
 [R1, ¬P3],
 [¬R1, P3],
 [¬R1, ¬P3],
 [T1, R1, P3],
 [T1, R1, ¬P3],
 [T1, ¬R1, P3],
 [T1, ¬R1, ¬P3],
 [¬T1, R1, P3],
 [¬T1, R1, ¬P3],
 [¬T1, ¬R1, P3],
 [¬T1, ¬R1, ¬P3]]

In [221]:
minimal_conflicts = []
conflicts = []
satisfiable = []
prime_implicants = []
implicants = []


for assignment in reduced_set:
    candidate = frozenset([p.get_variable(a.var.__str__()).get_assignment(a.val) for a in assignment])

    prune = False

    
    if candidate in children and is_valid_benchmark(candidate, sat) and not any(S.issubset(candidate) for S in prime_implicants):
        #print('children',children[frozenset([p.get_variable(a.var.__str__()).get_assignment(a.val) for a in assignment])])
        prime_implicants.append(candidate)


    elif len(candidate) == len(Vs) and \
        sat.check_consistency(candidate)[0] and \
        not any(S.issubset(candidate) for S in prime_implicants):
            prime_implicants.append(candidate)


    elif not sat.check_consistency(candidate)[0] and \
        not any(S.issubset(candidate) for S in minimal_conflicts):
            minimal_conflicts.append(candidate)


    elif any(S.issubset(candidate) for S in prime_implicants):
        implicants.append(candidate)


    elif any(S.issubset(candidate) for S in minimal_conflicts):
        conflicts.append(candidate)


    elif sat.check_consistency(candidate)[0]:
         satisfiable.append(candidate)


print('minimal_conflicts',minimal_conflicts)
print('conflicts',conflicts)
print('satisfiable',satisfiable)
print('prime_implicants',prime_implicants)
print('implicants',implicants)


minimal_conflicts [frozenset({¬R1, T1}), frozenset({T1, ¬P3}), frozenset({P3, R1, ¬T1})]
conflicts [frozenset({R1, T1, ¬P3}), frozenset({P3, ¬R1, T1}), frozenset({¬R1, T1, ¬P3})]
satisfiable [frozenset(), frozenset({T1}), frozenset({¬T1}), frozenset({R1}), frozenset({¬R1}), frozenset({P3}), frozenset({¬P3}), frozenset({R1, T1}), frozenset({P3, T1}), frozenset({R1, ¬T1}), frozenset({P3, ¬T1}), frozenset({P3, R1}), frozenset({R1, ¬P3}), frozenset({P3, ¬R1}), frozenset({¬R1, ¬P3})]
prime_implicants [frozenset({¬T1, ¬R1}), frozenset({¬T1, ¬P3}), frozenset({P3, R1, T1})]
implicants [frozenset({R1, ¬T1, ¬P3}), frozenset({P3, ¬T1, ¬R1}), frozenset({¬T1, ¬R1, ¬P3})]
